# Shioaji
![shioaji-logo](https://sinotrade.github.io/images/shioaji-logo-01.png)![sinopac-logo](https://www.sinotrade.com.tw/Images/logo.png)


[![PyPI - Status](https://img.shields.io/pypi/v/shioaji.svg?style=for-the-badge)](https://pypi.org/project/shioaji)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/shioaji.svg?style=for-the-badge)]()
[![PyPI - Downloads](https://img.shields.io/pypi/dm/shioaji.svg?style=for-the-badge)](https://pypi.org/project/shioaji)
[![Coverage](https://img.shields.io/badge/coverage%20-99%25-yellowgreen.svg?style=for-the-badge)]()
[![Binder](https://img.shields.io/badge/launch-Tutorial-ff69b4.svg?style=for-the-badge)](https://mybinder.org/v2/gh/Sinotrade/Sinotrade.github.io/master?filepath=tutorial%2Fshioaji_tutorial.ipynb)
[![doc](https://img.shields.io/badge/docs%20-passing-orange.svg?style=for-the-badge)](https://sinotrade.github.io/)
[![Telegram](https://img.shields.io/badge/chat-%20on%20telegram-blue.svg?style=for-the-badge)](https://t.me/joinchat/973EyAQlrfthZTk1)

Shioaji is a trading API provided by Sinopac that offers a comprehensive and user-friendly platform for accessing the Taiwan financial markets. With Shioaji, you can trade a variety of financial instruments including stocks, futures, and options using your favorite Python packages such as numpy, scipy, pandas, pytorch, or tensorflow to build your own custom trading models. The platform is easy to use and intuitive, with advanced charting tools, real-time market data, and a customizable interface that allows you to tailor your trading experience to your specific needs. Shioaji is fast and efficient, with a high-performance core implemented in C++ and using FPGA event broker technology, and it is the first Python trading API in Taiwan that is compatible with Linux and Mac, making it a truly cross-platform solution. Whether you are a beginner looking to get started in the world of trading or an experienced trader looking for a more powerful platform, Shioaji has something to offer. [Sign up for a free account today and start trading with confidence.](https://sinotrade.github.io/tutor/prepare/open_account)

## [Quickstarts](https://sinotrade.github.io/quickstart/)


## Getting Started

In [ ]:
!pip install shioaji

### Just import Our api like other popular python library to get start

In [ ]:
import shioaji as sj
from shioaji import constant

### Use Shioaji object to setup setting and login

In [ ]:
api = sj.Shioaji(simulation=True)

## ⚠️Apply for an API token on the [Sinopac](https://www.sinotrade.com.tw/newweb/PythonAPIKey/)⚠️

In [ ]:
api_key = "your api key"
secret_key ="your secret key"

In [ ]:
api.login(api_key=api_key, secret_key=secret_key)

### List all your account

In [ ]:
api.list_accounts()

### Activate your cetifacation to start ordering
❗❗❗ activate_ca is not necessary when using the **simulation** environment.

In [ ]:
# api.activate_ca(ca_path='../ca/Sinopac.pfx', ca_passwd='SCCEIEFAJA', person_id=person_id)

### Contract object for subscribe quote and place order

### You can obtain **contracts** in the following ways.

### Stock

In [ ]:
# For example, to obtain contract of TSMC (symbol 2330)
api.Contracts.Stocks.TSE.TSE2330
api.Contracts.Stocks['2330']

### Futures

In [ ]:
# For example, to obtain contract of TXF (symbol TXF)
api.Contracts.Futures.TXF.TXF202301
api.Contracts.Futures['TXFA3']

### Futures - R1、R2  the ticker symbol refers to the near month of expiration
**⚠️Currently only supports historical quotes query ⚠️**
- "R1" is referred to as the "current month" contract. 
- "R2" is referred to as the "next month" contract.

In [ ]:
api.Contracts.Futures.TXF.TXFR1
api.Contracts.Futures.TXF['TXFR1']

### Quote Api

### Prepare your callback, now just keep it simple
official document
- [Streaming Data for Stock](https://sinotrade.github.io/tutor/market_data/streaming/stocks/)
- [Streaming Data for Futures](https://sinotrade.github.io/tutor/market_data/streaming/futures/)

In [ ]:
# For example, subscribe to the tick for TSMC (symbol 2330)
from shioaji import Exchange, TickSTKv1, TickFOPv1
from shioaji import constant
 
@api.on_tick_stk_v1()
def quote_callback(exchange: Exchange, tick:TickSTKv1):
    print(f"Exchange: {exchange}, Tick: {tick}")

api.quote.subscribe(
    api.Contracts.Stocks["2330"], 
    quote_type = constant.QuoteType.Tick,
    version = constant.QuoteVersion.v1
)

In [ ]:
# For example, subscribe to the tick for Taiwan Stock Exchange Capitalization Weighted Stock Index (TAIEX) (symbol TXF)
from shioaji import Exchange, TickSTKv1, TickFOPv1
from shioaji import constant

from shioaji import TickFOPv1, Exchange

@api.on_tick_fop_v1()
def quote_callback(exchange:Exchange, tick:TickFOPv1):
    print(f"Exchange: {exchange}, Tick: {tick}")

api.quote.subscribe(
    api.Contracts.Futures.TXF['TXF202301'],
    quote_type = constant.QuoteType.Tick, # or 'tick'
    version = constant.QuoteVersion.v1, # or 'v1'
)

### Making Order object to place order

In [ ]:
from shioaji import constant

### Place Order - Stock

In [ ]:
TSE2890 = api.Contracts.Stocks['2890']

sample_order = api.Order(price=15.1,
                         quantity=2,
                         action=constant.Action.Buy,
                         price_type=constant.StockPriceType.LMT,
                         order_type=constant.OrderType.ROD,
                         daytrade=True,
                         account=api.futopt_account,
                        )

# just pass Order object to place_order fuction to place order then will get the Trade object return                        
trade = api.place_order(TSE2890, sample_order)
trade

### Place Order - Futures

In [ ]:
sample_order = api.Order(price=14000,
                         quantity=5,
                         action=constant.Action.Buy,
                         price_type=constant.FuturesPriceType.LMT,
                         order_type=constant.OrderType.ROD,
                         octype=constant.FuturesOCType.Auto,
                         account=api.futopt_account,
                        )

# just pass Order object to place_order fuction to place order then will get the Trade object return                        
trade = api.place_order(TXFA3, sample_order)
trade

### Update the trade object status to get the trade information

In [ ]:
api.update_status()

### Obtain all **trade** objects

In [ ]:
api.list_trades()

### Cancel order of the trade

In [ ]:
trade = api.cancel_order(trade)
trade

## Communication
- [Chat](https://t.me/joinchat/YpbYTNlXfoViYzQ1): The most timely and recommended channel for communication
- [Discuss](https://sjapi.tw): general chat, online discussions, collaboration etc.
- [GitHub issues](https://github.com/Sinotrade/Shioaji/issues): bug reports, feature requests, install issues, RFCs, thoughts, etc.

## Releases and Contributing
Shioaji current state is Pre-Alpha, we expect no obvious bugs. Please let us know if you encounter a bug by [filing an issue](https://github.com/Sinotrade/Shioaji/issues).

We appreciate all suggestions. If you have any idea want us to implement, please discuss with us in [telegram](https://t.me/joinchat/YpbYTNlXfoViYzQ1).

## The Team
Shioaji is currently maintained by [ec666 team](https://github.com/orgs/Sinotrade/teams/shioaji_dev/members) with major contributions.